# Trading News and Corporate Actions with BERT

<a href="" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


<a href="" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Algo-trading on corporate actions by leveraging NLP. A replicationa and enhancement of the paper: *Trade the Event: Corporate Events Detection for News-Based Event-Driven Trading (Zhou et al., Findings 2021)*.

We will perform the following steps:
1. Domain adaptation for financial articles by finetuning a BERT model with Masked Language Model (MLM) training on financial news and encyclopedia data. *Zhou et al.* utilized human annotators to label news articles with an event. 
1. Bi-Level Event Detection: At Token-Level we detect events using a sequence labeling approach. At the higher Article-Level we will augment the corpus with 'CLS' token embedding which contains the the aggregate of all the article's embeddings, and concatenate it with the lower level tokens.
1. Recognize security Ticker, using string matching algorithm to recognize tickers within articles.
1. Create trading signals on the identified tickers.

```bibtex
@inproceedings{zhou-etal-2021-trade,
    title = "Trade the Event: Corporate Events Detection for News-Based Event-Driven Trading",
    author = "Zhou, Zhihan  and
      Ma, Liqian  and
      Liu, Han",
    editor = "Zong, Chengqing  and
      Xia, Fei  and
      Li, Wenjie  and
      Navigli, Roberto",
    booktitle = "Findings of the Association for Computational Linguistics: ACL-IJCNLP 2021",
    month = aug,
    year = "2021",
    address = "Online",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2021.findings-acl.186",
    doi = "10.18653/v1/2021.findings-acl.186",
    pages = "2114--2124",
}
```

In [2]:
%pip install matplotlib
%pip install tqdm
%pip install pip install transformers


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import warnings
warnings.filterwarnings("ignore")

# Transformers cannot use keras3
os.environ['TF_USE_LEGACY_KERAS'] = '1'

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    print('Running in Kaggle...')
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

    DATA_PATH = "/kaggle/input/uscorpactionnews"
    IN_KAGGLE = True
else:
    IN_KAGGLE = False
    DATA_PATH = "./data/"

Running in Kaggle...
/kaggle/input/uscorpactionnews/Event_detection/train.txt
/kaggle/input/uscorpactionnews/Event_detection/dev.txt
/kaggle/input/uscorpactionnews/Trading_benchmark/evaluate_news.json
/kaggle/input/uscorpactionnews/Domain_adapation/train.txt
/kaggle/input/uscorpactionnews/Domain_adapation/dev.txt


In [4]:
import numpy as np
import math
import shutil
import pandas as pd

from pathlib import Path
import re
import pickle
from copy import deepcopy

from tqdm import tqdm
import tensorflow as tf

if ('COLAB_TPU_ADDR' in os.environ and IN_COLAB) or (IN_KAGGLE and 'TPU_ACCELERATOR_TYPE' in os.environ):
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
elif len(tf.config.list_physical_devices('GPU')) > 0:
    strategy = tf.distribute.MirroredStrategy()
    gpus = tf.config.experimental.list_physical_devices('GPU')
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
    finally:
        print("Running on", len(tf.config.list_physical_devices('GPU')), "GPU(s)")
else:
    strategy = tf.distribute.get_strategy()
    print("Running on CPU")

print("Number of accelerators:", strategy.num_replicas_in_sync)
os.getcwd()

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2024-04-10 17:46:43.247761: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 17:46:43.247918: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 17:46:43.248025: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 17:46:43.248120: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 17:46:43.248200: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 17:46:43.248477: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-10 17:46:43.248575: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this 

INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

'/kaggle/working'

# Wrangling the Data

Our corpus will be processed and labelled to 11 types of corporate events: 
1. Acquisition(A)
1. Clinical Trial(CT)
1. Regular Dividend(RD)
1. Dividend Cut(DC)
1. Dividend Increase(DI)
1. Guidance Increase(GI)
1. New Contract(NC)
1. Reverse Stock Split(RSS)
1. Special Dividend(SD)
1. Stock Repurchase(SR)
1. Stock Split(SS).
1. No event (O)

Articles are structured as follows:

```json
'title': 'Title',
'text': 'Text Body',
'pub_time': 'Published datetime',
'labels': {
    'ticker': 'Security symbol',
    'start_time': 'First trade after article published',
    'start_price_open': 'The "Open" price at start_time',
    'start_price_close': 'The "Close" price at start_time',
    'end_price_nday': 'The "Close" price at the last minute of the following 1-3 trading day. If early than 4pm ET its the same day. Otherwise, it refers to the next trading day.', 
    'end_time_1-3day': 'The time corresponds to end_price_1day',
    'highest_price_nday': 'The highest price in the following 1-3 trading', 
    'highest_time_nday': 'The time corresponds to highest_price_1-3day',
    'lowest_price_nday': 'The lowest price in the following 1-3 trading day',
    'lowest_time_nday': 'The time corresponds to lowest_price_1-3day',
}
```

In [5]:
import tensorflow as tf
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.optimizers import Adam, AdamW
from transformers import TFBertModel, BertConfig, BertTokenizerFast, TFBertModel, BertTokenizer, TFBertForTokenClassification, BertConfig, create_optimizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

MODEL_DIR = 'google-bert/bert-base-cased'
MAX_LEN = 256
SPECIAL_TOKEN = '[CLS]'
NUM_LABELS = 12 # See Labels description above.
LEARN_RATE=5e-5
EPOCHS=3
BATCH_SIZE = 1024

### Tokenizing News Text

The text body is tokenized, a simple example is shown below, including how the '[CLS]' (classification problem token) token is leveraged. BERT's transformer inputs expect of shape [batch_size, seq_length] the following inputs:
- "input_ids": token ids of the input sequences.
- "attention_mask": has value 1 at the position of all input tokens present before padding and value 0 for the padding tokens.
- "token_type_ids": the index of the input that created the input token. The first input segment (index 0) includes the start-of-sequence token and its end-of-segment token. The second segment (index 1, if present) includes its end-of-segment token. Padding tokens get index 0.

Transformers use self-attention mechanisms represent interactions amongst tokens and their contextual information in the input sequence as a weighted-sum. With this mechanism higher layers of the network will aggregate information from all other tokens in the sequence, in our case '[CLS]' will have such information. 


In [6]:
tokenizer = BertTokenizer.from_pretrained(MODEL_DIR)
model = TFBertModel.from_pretrained(MODEL_DIR)

text = ["Further this conclusion is the fact that cumulative raw returns increase and exceed the market over the same period. When taken as a whole, the evidence suggests Cramer recommends “hot” stocks, lending credence to the Hot Hand Fallacy in this context. "]

sample_inputs = inputs = tokenizer.encode_plus(
    text,
    add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
    max_length=MAX_LEN,  # Maximum length for padding/truncation, adjust as needed
    padding='max_length', 
    return_tensors='tf', 
    truncation=True
)

outputs = model(sample_inputs['input_ids'])
hidden_state = outputs.last_hidden_state
embedding = hidden_state[:, 0, :] # Get the hidden state with all info.

sample_inputs

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

{'input_ids': <tf.Tensor: shape=(1, 256), dtype=int32, numpy=
array([[101, 100, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,

In [7]:
def read_wnut(file_path):
    """
    Reads a WNUT-format file to extract and separate tokens and their NER tags.
    The WNUT-format expects each token and its corresponding tag to be separated by a tab ('\t'),
    with each token-tag pair on a new line. Different documents or sentences are separated by
    a blank line.

    Args:
        file_path (str): The path to the WNUT-format file containing the annotated text.

    Returns:
        tuple: A tuple containing two lists:
            - token_docs: A list where each element is a list of tokens from a single document/sentence.
            - tag_docs: A list where each element is a list of tags corresponding to the tokens in the `token_docs`.
    """
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

def encode_tags(tags, encodings, tag2id):
    """
    Aligns NER tags with the tokenized input for models employing subword tokenization, such as BERT, that requires adjusting the original word-level tags to match the tokenized structure.
    Note the ignore value (-100, from  PyTorch's CrossEntropyLoss) used for tokens that should not be directly associated with a tag from the original dataset, such as special tokens added during tokenization ([CLS], [SEP], [PAD]).

    Args:
        tags (list of list of str): The original list of tag sequences for each document.
        encodings (transformers.BatchEncoding): The tokenized input data with `offset_mapping` included.
        tag2id (dict): A mapping from tag names to unique integer IDs.

    Returns:
        list of list of int: A list of adjusted label sequences for each document, aligned with the tokenized input.
    """
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        doc_enc_labels = np.ones(len(doc_offset), dtype=int) * -100
        arr_offset = np.array(doc_offset)
        max_len = len(doc_enc_labels[(arr_offset[:, 0] == 0) & (arr_offset[:, 1] != 0)])
        doc_enc_labels[(arr_offset[:, 0] == 0) & (arr_offset[:, 1] != 0)] = doc_labels[:max_len]
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

def generate_sequence_labels(tags_list, tag2id, num_labels):
    """
    Generates sequence labels for a list of tag sequences.

    Args:
        tags_list (list of list of str): List of tag sequences.
        tag2id (dict): Mapping from tag names to unique integer IDs.
        num_labels (int): Number of unique labels in the dataset.

    Returns:
        list of list of int: Sequence labels for each tag sequence.
    """
    seq_labels = []
    for tags in tqdm(tags_list, desc="generate_sequence_labels"):
        tag_set = set(tags)
        current_label = np.zeros([num_labels])
        if len(tag_set) == 1:
            current_label[tag2id['O']] = 1
        else:
            tag_set.discard('O')
            for tag in tag_set:
                current_label[tag2id[tag]] = 1
        seq_labels.append(list(current_label))
    return seq_labels

def load_and_cache_dataset(path=DATA_PATH, model=MODEL_DIR, max_len=MAX_LEN, num_labels=NUM_LABELS, cache=True):
    """
    Preprocesses and caches the NER dataset for training with a BERT model, including ID maps.
    
    Args:
        path (str): Path to the dataset directory.
        model (str): Directory of the pretrained BERT model to use for tokenization.
        max_len (int): Maximum sequence length for tokenization.
        num_labels (int): Number of unique labels in the dataset.
        cache (bool): Whether to cache output, defaults to True.

    Returns:
        Tuple: Processed training and testing encodings, sequence labels, NER labels, tag2id, and id2tag.
    """
    cache_file = os.path.join(path if not IN_KAGGLE else ".", f'cached_train_test_{max_len}.pkl')
    print(f"{path}")

    if cache and os.path.exists(cache_file):
        print("Loading dataset from cache")
        with open(cache_file, 'rb') as f:
            loaded_data = pickle.load(f)
            return loaded_data  # This already includes tag2id and id2tag if the cache exists.

    # Your existing setup for tokenizer, reading data, etc.
    tokenizer = BertTokenizerFast.from_pretrained(model)
    train_ner_texts, train_ner_tags = read_wnut(os.path.join(path, 'Event_detection/train.txt'))
    test_ner_texts, test_ner_tags = read_wnut(os.path.join(path, 'Event_detection/dev.txt'))
    tags = deepcopy(train_ner_tags)
    tags.extend(test_ner_tags)

    unique_tags = list(set(tag for doc in train_ner_tags for tag in doc))
    tag2id = {tag: id for id, tag in enumerate(sorted(unique_tags))}
    print(f"Tags to identify: {tag2id}")
    id2tag = {id: tag for tag, id in tag2id.items()}

    # No changes needed below this line, except for including tag2id and id2tag in the data_to_save and return statements.
    train_seq_labels = generate_sequence_labels(train_ner_tags, tag2id, num_labels)
    test_seq_labels = generate_sequence_labels(test_ner_tags, tag2id, num_labels)

    train_encodings = tokenizer(train_ner_texts, is_split_into_words=True, return_offsets_mapping=True,
                                padding='max_length', truncation=True, max_length=max_len)
    test_encodings = tokenizer(test_ner_texts, is_split_into_words=True, return_offsets_mapping=True,
                               padding='max_length', truncation=True, max_length=max_len)

    train_ner_labels = encode_tags(train_ner_tags, train_encodings, tag2id)
    test_ner_labels = encode_tags(test_ner_tags, test_encodings, tag2id)
    train_encodings.pop("offset_mapping")
    test_encodings.pop("offset_mapping")

    if cache:
        data_to_save = (train_encodings, train_seq_labels, train_ner_labels, test_encodings, test_seq_labels,
                        test_ner_labels, tag2id, id2tag)
        with open(cache_file, 'wb') as f:
            pickle.dump(data_to_save, f)

    return train_encodings, train_seq_labels, train_ner_labels, test_encodings, test_seq_labels, test_ner_labels, tag2id, id2tag


train_encodings, train_seq_labels, train_ner_labels, test_encodings, test_seq_labels, test_ner_labels, tag2id, id2tag =  load_and_cache_dataset(DATA_PATH)

input_ids = np.array(train_encodings['input_ids'])
attention_mask = np.array(train_encodings['attention_mask'])
token_type_ids = np.array(train_encodings['token_type_ids']) if 'token_type_ids' in train_encodings else None
labels = np.array(train_seq_labels)
ner = np.array(train_ner_labels)
print("input_ids shape:", input_ids.shape)
print("attention_mask shape:", attention_mask.shape)
if token_type_ids is not None:
    print("token_type_ids shape:", token_type_ids.shape)
else:
    print("token_type_ids not present")

print("seq_labels shape:", labels.shape)
print("ner_labels shape:", ner.shape)

/kaggle/input/uscorpactionnews
Loading dataset from cache
input_ids shape: (7770, 256)
attention_mask shape: (7770, 256)
token_type_ids shape: (7770, 256)
seq_labels shape: (7770, 12)
ner_labels shape: (7770, 256)


In [8]:
unique, counts = np.unique(ner, return_counts=True)

df_tags = pd.DataFrame({'Tag ID': unique, 'Tag': (id2tag[id] if id in id2tag else 'UNK' for id in unique),'Count': counts})
df_sorted = df_tags.sort_values(by='Count', ascending=False).reset_index(drop=True)
df_sorted

,Tag ID,Tag,Count
0,11,O,1382863
1,-100,UNK,526415
2,5,I-NC,21218
3,1,I-CT,13231
4,9,I-SR,12583
5,0,I-A,8889
6,6,I-RD,7656
7,3,I-DI,6634
8,2,I-DC,2417
9,10,I-SS,2011


### Attention is all you Need

Vaswani et. al seminal paper *Attention is All You Need*, made self-attention and transformers mainstream.

Self-attention, calculates the relevance of each word in a sentence to every other word. This is done through queries (Q=XW^Q), keys (K=XW^K), and values (V=XW^V) transformed by a learned weight matrix (W) from the input embeddings (X). The attention score between two words is computed by taking the dot product of their queries and keys, followed by a softmax:

$ \text{A}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V $

Where:
- Q represents the queries matrix of current items,
- K represents the keys matrix of items to compare against in the input sequence,
- V represents the values matrix, which are the dot product comparisons between Q and K,
- d_k represents the dimension of the keys and queries,
- A are the Attention wieghts.

In addition, the word embeddings will contain contextual information (dot poduct of A and V), represented as position added to the embedding. This plus the attention wieghts, captures dependencies and relationships.

## BERT Classifier

Built on top of a pretrained BERT (Bidirectional Encoder Representations from Transformers).BERT is an industry tested transformer-based model, pre-trained on a large corpus of text to generate contextual embeddings for input sequences.

We will use a small pre-trained cased base model with 12-layers + 768-hidden, 12-heads , and 110M parameters. This is the base model used in *Zhou et al. (2021)*. Later in the article, we will use larger BERT models that are more resource demanding to fine-tune. Each model has its own preprocesser, as text inputs need to be converted to token IDs.

The architecture can be summarized in 3 componets:
1. Input embeddings, attention masks and ID types for the preTrained BERT model. Bert applies transformer blocks with self-attention (attention captures language structures). The model outputs embedding sequences (last layer from BERT NxH) and a pooled summary derived from the first 'CLS' token(a 1XH vector).
1. The sequence outputs (NxH vector) is passed through dense layers and dropouts for the first NER classification, this maps the high-DIM outputs to logits. Padding of unknown tokens helps the model focus on the tasks.
1. NER logits are flattened and concatenated with the pooled summaries to form a new feature vector (NxH + H). The vector is passed again through dense and dropout layers to classify the event as one of the 11 identified (O is ignored).

In [ ]:
def build_ner_model(bert_model, config, max_seq_length, num_labels):
    input_ids = Input(shape=(max_seq_length,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(max_seq_length,), dtype=tf.int32, name='attention_mask')
    token_type_ids = Input(shape=(max_seq_length,), dtype=tf.int32, name='token_type_ids')
    
    bert_layer = TFBertModel.from_pretrained(bert_model, config=config)
    bert_outputs = bert_layer({
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'token_type_ids': token_type_ids
    })

    sequence_output = bert_outputs.last_hidden_state

    # NER Classification layers
    ner_dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)(sequence_output)
    ner_dense = tf.keras.layers.Dense(2048, activation='relu')(ner_dropout)
    ner_dropout_2 = tf.keras.layers.Dropout(config.hidden_dropout_prob)(ner_dense)
    ner_logits = tf.keras.layers.Dense(num_labels, activation='linear', name='ner_logits')(ner_dropout_2)

    model = tf.keras.Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=ner_logits)

    return model


input_ids = np.array(train_encodings['input_ids'])
attention_mask = np.array(train_encodings['attention_mask'])
token_type_ids = np.array(train_encodings['token_type_ids']) if 'token_type_ids' in test_encodings else None
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'token_type_ids': token_type_ids
    },
    train_ner_labels
))
train_dataset = train_dataset.shuffle(buffer_size=10000).cache().prefetch(buffer_size=tf.data.AUTOTUNE).batch(BATCH_SIZE)
strategy = tf.distribute.get_strategy()
with strategy.scope():
    config = BertConfig.from_pretrained(MODEL_DIR)
    config.num_labels = NUM_LABELS 
    model = build_ner_model(MODEL_DIR, config, MAX_LEN, config.num_labels)
    optimizer = AdamW(learning_rate=LEARN_RATE)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    
    model.compile(optimizer=optimizer, 
                  loss=SparseCategoricalCrossentropy(ignore_class=-100,from_logits=True), metrics=[accuracy])
    history = model.fit(train_dataset, epochs=EPOCHS)

model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/3


In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Assuming 'test_encodings' and 'test_ner_labels', 'test_seq_labels' are correctly prepared beforehand
input_ids = np.array(test_encodings['input_ids'])
attention_mask = np.array(test_encodings['attention_mask'])
token_type_ids = np.array(test_encodings['token_type_ids']) if 'token_type_ids' in test_encodings else None

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'token_type_ids': token_type_ids
    },
    test_ner_labels
))
test_dataset = test_dataset.shuffle(buffer_size=10000).cache().prefetch(buffer_size=tf.data.AUTOTUNE).batch(BATCH_SIZE)


@tf.function
def distributed_prediction_step(batch):
    inputs, labels = batch
    predictions = strategy.run(lambda inputs: model(inputs, training=False), args=(inputs,))
    return predictions, labels


def distributed_predict(dataset):
    all_predictions = []
    all_true_labels = []

    for batch in tqdm(dataset, desc="Predicting"):
        predictions, true_labels = distributed_prediction_step(batch)

        # Note: Assuming strategy.gather correctly gathers across replicas, adjusting for your specific setup
        predictions = np.argmax(strategy.gather(predictions, axis=0).numpy(), axis=-1)
        true_labels = strategy.gather(true_labels, axis=0).numpy()

        all_predictions.extend(predictions.flatten())
        all_true_labels.extend(true_labels.flatten())

    # Filter out '-100' used for ignored tokens/padding
    valid_indices = np.where(np.array(all_true_labels) != -100)
    valid_predictions = np.array(all_predictions)[valid_indices]
    valid_true_labels = np.array(all_true_labels)[valid_indices]

    return valid_predictions, valid_true_labels


ner_predictions, ner_true_labels = distributed_predict(test_dataset)

# Calculate metrics
print("Precision: {:.2f}".format(precision_score(ner_true_labels, ner_predictions, average='macro', zero_division=0)))
print("Recall: {:.2f}".format(recall_score(ner_true_labels, ner_predictions, average='macro', zero_division=0)))
print("F1-Score: {:.2f}".format(f1_score(ner_true_labels, ner_predictions, average='macro', zero_division=0)))

# Ensure label_map values are in the same order as your model's output layer
sorted_label_names = [id2tag[i] for i in sorted(id2tag)]
print("\nFull classification report:")
print(classification_report(ner_true_labels, ner_predictions, target_names=sorted_label_names, zero_division=0))

In [ ]:
import tensorflow as tf
from transformers import TFBertModel, BertConfig
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

def build_bert_for_bilevel_classification(bert_model_name, max_seq_length, num_labels, learning_rate=5e-5):
    """
    Builds a BERT model for bilevel classification tasks using TensorFlow's functional API.

    Args:
        bert_model_name (str): The pre-trained BERT model identifier.
        max_seq_length (int): Maximum sequence length for inputs.
        num_labels (int): Number of labels for the classification tasks.
        learning_rate (float): Learning rate for the optimizer.

    Returns:
        A TensorFlow model configured for bilevel classification.
    """
    # Load BERT model
    config = BertConfig.from_pretrained(bert_model_name)
    config.num_labels = num_labels
    bert = TFBertModel.from_pretrained(bert_model_name, config=config)

    # Define inputs
    input_ids = Input(shape=(max_seq_length,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_seq_length,), dtype=tf.int32, name="attention_mask")
    token_type_ids = Input(shape=(max_seq_length,), dtype=tf.int32, name="token_type_ids")

    # BERT outputs
    bert_outputs = bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    sequence_output = bert_outputs.last_hidden_state
    pooled_output = bert_outputs.pooler_output

    # NER branch
    ner_dropout = Dropout(config.hidden_dropout_prob)(sequence_output)
    ner_intermediate = Dense(2048, activation='relu')(ner_dropout)
    ner_dropout_2 = Dropout(config.hidden_dropout_prob)(ner_intermediate)
    ner_logits = Dense(num_labels, activation='softmax', name="ner_logits")(ner_dropout_2)

    # Sequence classification branch
    seq_concat = tf.reshape(ner_logits, shape=(-1, max_seq_length * num_labels))
    final_input = tf.concat([pooled_output, seq_concat], axis=-1)
    
    seq_dropout_1 = Dropout(config.hidden_dropout_prob)(final_input)
    seq_intermediate = Dense(2048, activation='relu')(seq_dropout_1)
    seq_dropout_2 = Dropout(config.hidden_dropout_prob)(seq_intermediate)
    seq_logits = Dense(num_labels - 1, activation='sigmoid', name="seq_logits")(seq_dropout_2)

    # Compile model
    model = Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=[ner_logits, seq_logits])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss={'ner_logits': 'sparse_categorical_crossentropy', 'seq_logits': 'binary_crossentropy'},
                  metrics={'ner_logits': 'sparse_categorical_accuracy', 'seq_logits': 'accuracy'})

    return model

MAX_LEN = 256
SPECIAL_TOKEN = '[CLS]'
NUM_LABELS = 12 # See Labels description above.

model = build_bert_for_bilevel_classification(MODEL_DIR, MAX_LEN, NUM_LABELS, LEARN_RATE)
model.summary()

In [ ]:
sample_inputs = inputs = tokenizer.encode_plus(
    text,
    add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
    max_length=MAX_LEN,  # Maximum length for padding/truncation, adjust as needed
    padding='max_length', 
    return_tensors='tf', 
    truncation=True
)
model(sample_inputs, training=False)

In [ ]:
@tf.function
def distributed_prediction_step(batch):
    def prediction_step(inputs):
        ner_logits, seq_logits = model(inputs, training=False)
        return ner_logits, seq_logits
    
    inputs, labels = batch
    ner_logits, seq_logits = strategy.run(prediction_step, args=(inputs,))
    return ner_logits, seq_logits, labels['ner_labels'], labels['seq_labels']

def distributed_predict(dataset):
    ner_true_labels = []
    ner_predictions = []
    seq_true_labels = []
    seq_predictions = []
    for batch in tqdm(dataset, desc="Predicting"):
        ner_logits, seq_logits, ner_labels, seq_labels = distributed_prediction_step(batch)
        
        # Gather across replicas
        ner_logits = strategy.gather(ner_logits, axis=0)
        seq_logits = strategy.gather(seq_logits, axis=0)
        ner_labels = strategy.gather(ner_labels, axis=0)
        seq_labels = strategy.gather(seq_labels, axis=0)

        ner_pred = np.argmax(ner_logits.numpy(), axis=-1)
        seq_pred = (seq_logits.numpy() > 0.5).astype(float)
        ner_true = ner_labels.numpy()
        seq_true = seq_labels.numpy()
        
        
        # Accumulate predictions and true labels
        ner_predictions.extend(ner_pred.flatten())
        ner_true_labels.extend(ner_true.flatten())
        seq_predictions.extend(seq_pred)
        seq_true_labels.extend(seq_true)

    return ner_predictions, seq_predictions, ner_true_labels, seq_true_labels

with strategy.scope():
    ner_logits, seq_logits, ner_labels, seq_labels = distributed_predict(test_dataset)


In [ ]:
valid_indices = np.nonzero(np.asarray(ner_labels) != -100)[0]
ner_true_labels_filtered = np.asarray(ner_labels)[valid_indices]
ner_predictions_filtered = np.asarray(ner_logits)[valid_indices]

ner_precision = precision_score(ner_true_labels_filtered, ner_predictions_filtered, average='macro', zero_division=0)
ner_recall = recall_score(ner_true_labels_filtered, ner_predictions_filtered, average='macro', zero_division=0)
ner_f1 = f1_score(ner_true_labels_filtered, ner_predictions_filtered, average='macro', zero_division=0)

seq_accuracy = accuracy_score(seq_logits, seq_logits)

print(f"NER Precision: {ner_precision}, NER Recall: {ner_recall}, NER F1: {ner_f1}")
print(f"Sequence Classification Accuracy: {seq_accuracy}")

In [ ]:
seq_logits

# Conclusion

TODO

## References

- [Zhou, Zhihan, Liqian Ma, and Han Liu. "Trade the event: Corporate events detection for news-based event-driven trading." arXiv preprint arXiv:2105.12825 (2021).](https://aclanthology.org/2021.findings-acl.186)
- [Hugging Face Transformers APIs](https://github.com/huggingface/transformers)
- [Hugging Face Model Repository and Spaces](https://huggingface.co/models)
- [Devlin, Jacob, et al. "Bert: Pre-training of deep bidirectional transformers for language understanding." arXiv preprint arXiv:1810.04805 (2018).](https://arxiv.org/abs/1810.04805)
- [Google Pre-trained BERT Models.](https://github.com/google-research/bert?tab=readme-ov-file)
- [Ashish Vaswani and Noam Shazeer and Niki Parmar and Jakob Uszkoreit and Llion Jones and Aidan N. Gomez and Lukasz Kaiser and Illia Polosukhin, "Attention is All You Need". NIPS (2017)](https://research.google/pubs/attention-is-all-you-need/)

## Github

Article and code available on [Github](https://github.com/adamd1985/news-based-event-driven_algotrading)

Kaggle notebook available [here]()

Google Collab available [here]()

## Media

All media used (in the form of code or images) are either solely owned by me, acquired through licensing, or part of the Public Domain and granted use through Creative Commons License.

## CC Licensing and Use

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.